In [5]:
from kafka import KafkaConsumer
import pyspark.sql.functions as F

In [129]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "twitter-data") \
  .load()

# df.printSchema()
df = df.selectExpr("CAST(value AS STRING)")

words = df.select(
   F.explode(
       F.split(df.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

In [130]:
# df = df.withColumn('test_col', F.lit(1))

In [138]:
# query = df \
#     .writeStream.outputMode("append") \
#     .format("console") \
#     .queryName("test1") \
#     .trigger(processingTime='2 seconds') \
#     .start()

query = wordCounts \
    .writeStream.outputMode("complete") \
    .format("console") \
    .start()
#     .option("path", "output_folder") \
#     .option("checkpointLocation", "checkpointLocationFolder") \

In [137]:
# query.awaitTermination()

In [122]:
# [q.stop() for q in spark.streams.active]
query.stop()

In [135]:
# query.status
spark.streams.active

[]